In [25]:
import csv
import os
from pathlib import Path

import numpy as np

from compare_genes import getScores
from xmlparse import loadRef, loadGeometryBases, getXmlScore, minXml
from score_strokes import alignStrokes
from exhaustive import exhaustScore

## Scoring Functions

In [26]:
# Obtaining scores through heuristic algorithm without getScores
def heuristicScores(algo, ref_char, data_dir):
    ref_g, ref_l, output_size = loadRef(ref_char, "Reference")
    g_data, _, base_data, stroke_sets, _, f_names= loadGeometryBases(data_dir, output_size)
    heuristic_scores = []
    for (gl, bases, stroke_set, f_name) in zip(g_data, base_data, stroke_sets, f_names):
        g, l = gl
        heuristic_alignment = np.array(algo(g, ref_g, l, ref_l))+1
        heuristic_xml = minXml(ref_char, bases, stroke_set, heuristic_alignment)
        heuristic_score = getXmlScore(heuristic_xml)
        heuristic_scores.append(heuristic_score)
    return heuristic_scores

# Obtaining scores through heuristic algorithm with getScores
def heuristicScoresShort(algo, ref_char, data_dir):
    heuristic_scores, _, marks = getScores(algo, ref_char, data_dir)
    return heuristic_scores, marks

# Obtaining scores through exhaustive search
def exhaustiveScores(ref_char, data_dir):
    _, _, output_size = loadRef(ref_char, "Reference")
    g_data, han_chars, base_data, _, _, f_names = loadGeometryBases(data_dir, output_size)
    exhaustive_scores = []
    for (gl, han_char, bases, f_name) in zip(g_data, han_chars, base_data, f_names):
        g, l = gl
        original_score = exhaustScore(ref_char, f_name, data_dir, force_refresh=True, save=False)
        exhaustive_scores.append(original_score)
    return exhaustive_scores

## Gene/Archetype Combos

In [27]:
data_dir = f'{str(Path.home())}/Stylus_Scoring_Generalization/NewGenes' # gene directory
ref_dir = f'{str(Path.home())}/Stylus_Scoring_Generalization/Reference' # archetype directory

# Retrieve all reference characters
ref_chars = []
dirs = [f.path for f in os.scandir(ref_dir) if f.is_dir()]
for d in dirs:
    ref_chars.extend(f.split(".")[0] for f in os.listdir(d))
ref_chars = list(filter(None, ref_chars))